In [1]:
from collections import defaultdict
import pickle
from datetime import datetime, timedelta

import pandas as pd

### Chicago
Data source for Chicago:

https://www.chicago.gov/city/en/sites/covid-19/home/latest-data.html

There is also a CSV at https://data.cityofchicago.org/Health-Human-Services/Daily-COVID-19-Cases-and-Deaths/kxzd-kd6a, but this appears to be based on reported dates for deaths, which is not clean.

Create a dict of the form {days from Feb 29: number of deaths}, and save it as a pandas DataFrame starting with Feb 1 to the end of the dict.

In [26]:
chi_death_dict = defaultdict(int, {16: 1, 17: 1, 18: 1, 19: 2, 21: 2,
                                   22: 2, 23: 2, 24: 2, 25: 5, 26: 2,
                                   27: 7, 28: 4, 29: 21, 30: 12,
                                   31: 16, 32: 24, 33: 14, 34: 25,
                                   35: 21, 36: 25, 37: 22, 38: 25,
                                   39: 34, 40: 26, 41: 30, 42: 33,
                                   43: 32, 44: 32, 45: 33, 46: 31,
                                   47: 30, 48: 36, 49: 36, 50: 34,
                                   51: 34, 52: 31, 53: 45, 54: 33,
                                   55: 49, 56: 35, 57: 46, 58: 31,
                                   59: 41, 60: 37, 61: 30, 62: 28,
                                   63: 44, 64: 36, 65: 41, 66: 46,
                                   67: 37, 68: 23, 69: 5})

In [27]:
chi_last_day = max(chi_death_dict.keys())

In [28]:
chi_index = [datetime(2020, 2, 1) + timedelta(days=day) for day in range(29 + chi_last_day)]
chi_deaths =[chi_death_dict[day - 28] for day in range(29 + chi_last_day)]

In [29]:
chi_death_df = pd.DataFrame(chi_deaths, index=chi_index, columns=['deaths'])

In [30]:
# Inspect that last few days
chi_death_df.tail()

,deaths
2020-05-04,41
2020-05-05,46
2020-05-06,37
2020-05-07,23
2020-05-08,5


In [31]:
chi_death_df.to_pickle('../data/chi_deaths.pickle')